In [13]:
import os
import numpy as np
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


OSError: [WinError 126] The specified module could not be found. Error loading "D:\python\pythonProject\.venv\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [2]:
# Constants
DATA_PATH = "../data/audio_speech_actors_01-24/"
MODEL_PATH = "../model/voice_emotion_model.pth"

In [3]:
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T, axis=0)
    return mfccs_processed

In [4]:
def load_data(data_path):
    features = []
    labels = []
    for actor_dir in os.listdir(data_path):
        actor_path = os.path.join(data_path, actor_dir)
        for file_name in os.listdir(actor_path):
            file_path = os.path.join(actor_path, file_name)
            mfccs = extract_features(file_path)
            label = int(file_name.split('-')[2]) - 1  # Emotion labels are in the 3rd position
            features.append(mfccs)
            labels.append(label)
    return np.array(features), np.array(labels)

In [5]:
# Load and preprocess data
features, labels = load_data(DATA_PATH)

In [7]:
# Import necessary libraries
from sklearn.preprocessing import LabelBinarizer

# Encode labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [14]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [17]:
import torch.nn as nn
# Define the model
class VoiceEmotionNN(nn.Module):
    def __init__(self):
        super(VoiceEmotionNN, self).__init__()
        self.fc1 = nn.Linear(40, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 8)  # 8 emotion classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

OSError: [WinError 126] The specified module could not be found. Error loading "D:\python\pythonProject\.venv\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
model = VoiceEmotionNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training the model
num_epochs = 50  # Adjust number of epochs as needed
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:

# Save the model
torch.save(model.state_dict(), MODEL_PATH)


In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy * 100:.2f}%')